<a href="https://colab.research.google.com/github/AlexandreXavierMartins/PulpModels/blob/main/modelo_CSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 27.0 MB/s 


In [ ]:
import gurobipy as grb

def modeloPcut(tam,lar,D):
  
  I = range(len(tam))
  K = range(len(lar))

  # Criação do objeto modelo
  modelo  = grb.Model('pcut')

  #criação de variáveis
  x = {} # x é um dicionário
  for i in I:
    for k in K:
      x[i,k] = modelo.addVar(name = f'x[{i},{k}]',vtype= grb.GRB.INTEGER)

  y = {}
  for k in K:
    y[k] = modelo.addVar(name = f'y[{k}]',vtype= grb.GRB.BINARY)

  #atualização do modelo
  modelo.update()

  #Restrições
  #demanda
  for i in I:
    modelo.addConstr(grb.quicksum(x[i,k] for k in K) == D[i])

  #oferta
  for k in K:
    modelo.addConstr(grb.quicksum(tam[i]*x[i,k] for i in I) <= lar[k]*y[k])

  #objetivo
  Qxy = grb.quicksum(y[k] for k in K)

  modelo.setObjective(Qxy, sense=grb.GRB.MINIMIZE)

  return modelo

tamanho = [1380, 1520, 1560, 1710, 1820, 1880, 1930, 2000, 2050, 2100, 2140, 2150, 2200]
demanda = [22, 25, 12, 14, 18, 18, 20, 10, 12, 14, 16, 18, 20]
print(len(tamanho),len(demanda))
largura = [5600]*80

m =  modeloPcut(tamanho, largura, demanda)
m.setParam('Heuristics', 0)
m.optimize()
